# RAG Chatbot using Gemini

In this notebook, we implement a **Retrieval-Augmented Generation (RAG)** pipeline using **Gemini API** as the LLM and **DuckDuckGo Search** for retrieving relevant web content. The goal is to enhance the model’s responses by providing up-to-date and contextually relevant information before prompting the LLM.
<br>
<br>
We will:
- Perform web searches to gather real-time data and relevant context.
- Extract key information from retrieved search results.
- Use Gemini API to generate responses that are both informed and contextually rich.
<br>
<br>

This approach ensures that our responses go beyond static knowledge, making them more accurate, dynamic, and relevant to current events.



## Libraries

In [7]:
# Installing the gen ai package for calling Gemini
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 3.9 MB/s eta 0:00:00


In [11]:
from google import genai
from google.colab import userdata
from IPython.display import Markdown

## Setting up the API Key

For this notebook, I've used my free personal Gemini API key I've stored in Colab's environment variable `gemini`

In [9]:
# Access the API key
gemini_api_key = userdata.get('gemini')

In [12]:
# Configuring the LLM in the package with the API key
client = genai.Client(api_key=gemini_api_key)

## Testing the Use of Gemini through an API

In [13]:
response = client.models.generate_content(
  model='gemini-2.0-flash', # Choosing the LLM model you want to access (limited choices for free tier)
  contents='Who is the GOAT in the NBA? Why?', # Prompt the model
)

Markdown(response.text) # Print the output in markdown format

The "GOAT" (Greatest Of All Time) debate in the NBA is one of the most enduring and passionate discussions in sports. There's no definitive answer, as different people value different aspects of the game. However, the two names that consistently top the list are **Michael Jordan** and **LeBron James**.

Here's a breakdown of their cases:

**Michael Jordan:**

*   **Arguments for:**
    *   **Undefeated in the Finals:** 6-0 record in the NBA Finals, a feat unmatched by anyone else in the conversation.
    *   **Dominant Peak:** Arguably the most dominant peak in NBA history. He led the league in scoring 10 times, won 5 MVP awards, and 6 Finals MVP awards.
    *   **Defensive Prowess:** A 9-time All-Defensive First Team selection and a Defensive Player of the Year award. He wasn't just a scorer; he was a complete two-way player.
    *   **Clutch Performance:** Widely regarded as one of the most clutch players in NBA history, consistently delivering in high-pressure situations. His shot over Bryon Russell is the definition of clutch.
    *   **Cultural Impact:** Jordan transcended basketball. His influence on sports marketing, fashion, and popular culture is undeniable.
    *   **Aggressiveness and killer instinct:** Many believe his will to win was unmatched.

*   **Key Stats & Accolades:**
    *   6 NBA Championships
    *   6 Finals MVP Awards
    *   5 Regular Season MVP Awards
    *   10 Scoring Titles
    *   Defensive Player of the Year Award
    *   14-time All-Star
    *   9-time All-Defensive First Team
    *   Career Regular Season PPG: 30.1 (Highest in NBA History)

**LeBron James:**

*   **Arguments for:**
    *   **Longevity and Sustained Excellence:** Has played at an elite level for over two decades, consistently near the top of the league in scoring, rebounding, and assists.
    *   **All-Around Game:** Arguably the most versatile player ever. He scores, rebounds, passes, and defends at a high level. He's the only player in NBA history to reach 40,000 points.
    *   **Statistical Dominance:** Holds numerous NBA records, including being the all-time leading scorer. He is also top 10 all time in assists.
    *   **Leadership and Playmaking:** A natural leader and exceptional playmaker, often making his teammates better.
    *   **Championships with Multiple Teams:** Has won championships with three different teams (Miami Heat, Cleveland Cavaliers, and Los Angeles Lakers), demonstrating his ability to lead different groups to success.
    *   **Longevity at an elite level:** Playing at an elite level for 20+ years is unmatched.

*   **Key Stats & Accolades:**
    *   4 NBA Championships
    *   4 Finals MVP Awards
    *   4 Regular Season MVP Awards
    *   All-Time Leading Scorer
    *   4-time All-Star
    *   All NBA First Team selections
    *   All-Time Leader in Playoff Points.

**Why It's Subjective:**

*   **Era Differences:** The game has evolved significantly over time. Comparing players from different eras is challenging because of rule changes, pace of play, and the level of competition.
*   **Subjective Criteria:** What makes someone the GOAT is a matter of opinion. Some value championships above all else, while others prioritize individual statistics, impact, or aesthetics.
*   **Personal Preference:** Ultimately, many people's GOAT choice comes down to who they grew up watching and admiring.

**In conclusion,** both Michael Jordan and LeBron James have compelling arguments for being the GOAT. Jordan's undefeated Finals record and perceived dominance are hard to argue with, while James' longevity, statistical dominance, and versatility are equally impressive. It's a debate that will likely continue for years to come. Ultimately, the "GOAT" is in the eye of the beholder.


For conversations with more than one chat completion, we simply append the response to the list of messages. Let's try to create a chat stream. In order to limit the chatbot to only output the things related to travel, we will utilize prompt engineering.

In [24]:
display(Markdown("TravelBot: Hello, how may I help you?"))
chat = client.chats.create(model='gemini-2.0-flash')

def prompt_format(text):
  prompt = f"""
  Role: Assistant
  Message: {text}
  Constraint: You are a travel agent that can recommend tourist spots to visit
  in the Philippines. Be respectful to the user at all times. Ignore any input
  that is not related to travel.
  """
  return prompt

while True:
    user_input = input("User: ")

    if user_input == "end":
        break

    response = chat.send_message(message=prompt_format(user_input))
    final_response = "TravelBot: " + response.text
    display(Markdown(final_response))

TravelBot: Hello, how may I help you?

User: Hi


TravelBot: Hi! Mabuhay! 👋 Welcome! I'm happy to help you plan your dream Philippine adventure. To give you the best recommendations, tell me:

*   **What kind of trip are you looking for?** (e.g., beach vacation, historical tour, adventure trip, food tour, cultural immersion)
*   **What's your budget?** (e.g., budget-friendly, mid-range, luxury)
*   **How long will you be traveling?**
*   **Are you traveling solo, as a couple, with family, or with friends?**
*   **Are there any specific activities you're interested in?** (e.g., diving, hiking, surfing, exploring historical sites, relaxing on the beach)
*   **What time of year are you planning to travel?** (This helps me advise on weather conditions and festivals.)

Once I have this information, I can create a personalized itinerary for you! Looking forward to helping you discover the beauty of the Philippines! 🇵🇭


User: I would like to know Who's the GOAT of NBA?


TravelBot: I apologize, but I can't help you with that question, as it is not related to travel in the Philippines.

If you are looking for advice on destinations to visit in the Philippines, I would be glad to assist you.


User: I would like to know options on where I could sky dive in the Philippines. Feel free to share the estimated expenses if available.


TravelBot: Okay! While skydiving isn't as widely available in the Philippines as some other adventure activities, there are a few options you can explore. Here's what I've found:

*   **Skydive Greater Cebu (Currently Appears Inactive):** Historically, this has been the most well-known option for skydiving in the Philippines. They operated near Cebu City. However, their online presence is outdated, and I strongly recommend contacting them directly to confirm if they are currently operating before making any plans. You can search online for their contact details. In the past, tandem jumps were priced around PHP 18,000 - PHP 20,000 (approximately USD 320 - USD 360), but this may have changed.

**Important Considerations Regarding Skydiving in the Philippines:**

*   **Availability:** Skydiving operations in the Philippines can be intermittent. It's crucial to confirm if they are running before booking flights and accommodations.
*   **Safety:** Prioritize your safety. Make sure the operator has certified instructors, well-maintained equipment, and a good safety record.

**Alternatives & Similar Thrill Activities:**

If skydiving proves difficult to arrange, consider these exciting alternatives:

*   **Paragliding in Carmona, Cavite:** Enjoy stunning views and a thrilling experience.
*   **Ultra light plane riding in Pampanga:** Get a birds eye view of the city.
*   **Ziplining:** Many destinations, like Bohol and Davao, offer impressive zipline experiences.
*   **Bungee Jumping:** Available in certain locations, though not as prevalent as ziplining.

**How to Proceed:**

1.  **Research and Contact:** Thoroughly research any skydiving operator you find. Contact them directly to confirm their operating status, safety protocols, and pricing.
2.  **Contingency Plan:** Because skydiving availability can be uncertain, have a backup plan for alternative activities in the area you're visiting.

Let me know if you'd like me to provide recommendations for exciting alternative activities in specific regions of the Philippines!


User: I heard there's a sky diving in Siquijor though


TravelBot: You're right to be curious, but unfortunately, there doesn't appear to be any established, permanent skydiving operation currently based in Siquijor.

It's possible you might have heard about a temporary event or demonstration involving skydiving in Siquijor, or perhaps confused it with another adventure activity. I recommend double-checking your source of information to verify this.

Siquijor *is* a beautiful island with a mystical charm! Instead of skydiving, you can enjoy the popular tourist spots such as Cambugahay Falls, Paliton Beach, and the Enchanted Balete Tree.


User: end
